In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Data Consumption

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@dbxdev.dfs.core.windows.net/products")
display(df)

In [0]:
df = df.drop('_rescued_data')
display(df)

In [0]:
df.createOrReplaceGlobalTempView("products")

# Functions

In [0]:
%sql
CREATE OR REPLACE FUNCTION azuredb_catalog.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
    

select product_id,price, azuredb_catalog.bronze.discount_func(price) as discounted_price
FROM global_temp.products

In [0]:
df = df.withColumn("discounted_price", expr("azuredb_catalog.bronze.discount_func(price)"))
display(df)


In [0]:
%sql

CREATE OR REPLACE FUNCTION azuredb_catalog.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS 
$$ 
    return p_brand.upper()
$$


In [0]:
%sql

SELECT product_id, brand, azuredb_catalog.bronze.upper_func(brand) as brand_upper
FROM global_temp.products

# Data Persistence

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path", "abfss://silver@dbxdev.dfs.core.windows.net/products")\
    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS azuredb_catalog.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@dbxdev.dfs.core.windows.net/products'